In [16]:
import time
from qwikidata.entity import WikidataItem
from qwikidata.json_dump import WikidataJsonDump
from qwikidata.utils import dump_entities_to_json
from IPython.display import JSON
from pymongo import MongoClient
from joblib import Parallel, delayed

In [22]:
# create an instance of WikidataJsonDump
wjd_dump_path = "/storage/wikidata/wikidata-20220103-all.json.gz"
wjd = WikidataJsonDump(wjd_dump_path)


In [4]:
client = MongoClient()

In [5]:
#busco los ids de los que ya tengo para sacar los tipos de institución y luego hacer el corte
kahi_wdata_cursor = client["kahi_dev"]["affiliations"].find({"external_ids.source":"wikidata"},{"_id":0,"external_ids":1})
kahi_wdata_ids=[]
for i in kahi_wdata_cursor:
    for j in i["external_ids"]:
        if j["source"] == "wikidata":
            kahi_wdata_ids.append(j["id"])
            break


In [26]:
kahi_wdata_ids=list(set(kahi_wdata_ids))
len(kahi_wdata_ids)

18825

In [27]:
%%time
inst=[]
# create an iterable of WikidataItem representing politicians
for ii, entity_dict in enumerate(wjd):
    if entity_dict["id"] in kahi_wdata_ids:
        inst.append(entity_dict)

    if len(inst) == len(kahi_wdata_ids):
        break


CPU times: user 9h 24min 4s, sys: 1min 39s, total: 9h 25min 43s
Wall time: 9h 26min 18s


In [28]:
len(inst)

18606

In [17]:
backend = 'multiprocessing'
out = Parallel(n_jobs=20, backend=backend,verbose=10)(delayed(process_one)(ints) for ints in kahi_wdata_ids)

[Parallel(n_jobs=20)]: Using backend MultiprocessingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:  1.7min
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed: 118.8min
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed: 414.1min
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed: 538.5min
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed: 654.8min
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed: 779.0min


KeyboardInterrupt: 

In [16]:
len(institutions)

1

In [17]:
JSON(institutions)

<IPython.core.display.JSON object>